<center> <img src="profitroll.png">

# <center><span style="font-size: 50px; color: blue;">PROFITROLL BACKUP DEMO</span></center>

<center><span style="font-size: 25px; color: purple;">This notebook is an advanced tutorial for users already familiar with <b><i>profitroll<i/></b> basic use. If you discover profiteroll, you might want to begin with the <i>profiteroll_demo.ipynb<i/> notebook </span></center>
<p></p>

In [ ]:
from profitroll.core.grid import Grid
from profitroll.core.state import State
from profitroll.core.simulation import Simulation
from profitroll.test.test_cases import v_stripe_test, bubble_test, gaussian_test

# Scientific methods
from profitroll.methods.pseudo_spectral_wind import pseudo_spectral_wind
from profitroll.methods.wrap_advection_step_3P import wrap_advection_step_3P
from profitroll.methods.wrap_wv import wrap_wv
from profitroll.methods.end_pop import end_pop

# Simulation parameters

In [ ]:
Lx = 2048e3
Ly = 1024e3
Nx = 256
Ny = 128


T = 3*3600 # Complete simulation is no more very long
dt = 300
Nt = int(T//dt)


dX = Nx//8  # used to shape the initial v-stripe data
dY = Ny//15

nb_state = 2  # number of instants in initial data

# Simulation parameters and building

In [ ]:
methods = [pseudo_spectral_wind,
           wrap_advection_step_3P,wrap_wv,end_pop]

methods_kwargs = [{},
                  {'alpha_method' : 'damped_bicubic',
                   'order_alpha' : 2, 
                   'F_method' : 'damped_bicubic'},
                  {'alpha_method' : 'damped_bicubic',
                   'order_alpha' : 2, 
                   'F_method' : 'damped_bicubic'},
                  {}]

output_folder = 'output_backup_test'
save_rate = 2
backup_rate = 10
verbose = 1 # displaying level, usefull to inspect what's going wrong

In [ ]:
# Creation of the test case
initialCDF = v_stripe_test('initial.nc', Lx, Ly, Nx, Ny, dt, nb_state, dX, dY)

In [ ]:
# Creation of the simulation object
mySim = Simulation(initialCDF,
                   methods, 
                   methods_kwargs,
                   output_folder,
                   verbose=verbose, 
                   name='testfb')

In [ ]:
# Run the simulation
mySim.run(T, Nt, save_rate, backup_rate, first_run=True) 

### if you want to extend the simulation a little bit...

We will extend the first run (3h of simulation) with a new one (5h)

In [ ]:
T2 = 5*3600 
Nt2 = int(T2//dt)

$\textbf{Try interrupting this run ! (interrupt the kernel during simulation)}$ You will see how to launch a new simulation from the backup file to continue the simulation later

In [ ]:
save_rate = 1
backup_rate = 6
mySim.run(T2, Nt2, save_rate, backup_rate, first_run=False)

# netCDF results can easily be analyzed...

In [ ]:
from netCDF4 import Dataset
import numpy as np

In [ ]:
resultsCDF = Dataset(output_folder + '/results_testfb.nc', 'r', format='NETCDF4', parallel=False)
backupCDF = Dataset(output_folder + '/backup_testfb.nc', 'r', format='NETCDF4', parallel=False)

To see the saved times and the last backup time :

In [ ]:
print(resultsCDF['t'][:].data)
print(backupCDF['t'][:].data)

To see the parameters of the different runs :

In [ ]:
print(resultsCDF.T)
print(resultsCDF.Nt)
print(resultsCDF.save_rate)
print(resultsCDF.backup_rate)

Don't forget to close the datasets

In [ ]:
resultsCDF.close()
backupCDF.close()

# Checking backup start

If the last run has been interrupted, you'll see how to continue the simulation with the backup file and the result file (to copy the states saved from the beginning to the last backup). If the result file is corrupted or can't be used, you won't retrive the previous data, but if you're only interested in the end of the simulation you can begin the new run at the last backup thanks to this.

In [ ]:
backupCDF = Dataset(output_folder + '/backup_testfb.nc', 'r', format='NETCDF4', parallel=False)
pre_resultCDF = Dataset(output_folder + '/results_testfb.nc', 'r', format='NETCDF4', parallel=False)

In [ ]:
print(backupCDF.methods)

In [ ]:
mySim_fromb = Simulation.frombackup(backupCDF,
                                    methods,
                                    methods_kwargs,
                                    output_folder,
                                    resultCDF=pre_resultCDF,
                                    name='testfb_fb',
                                    verbose=2)

In [ ]:
mySim_fromb.run(#tocomplete,#tocomplete,#tocomplete,#tocomplete,first_run=True)

In [ ]:
resultsCDF = Dataset(output_folder + '/results_testfb_fb.nc', 'r', format='NETCDF4', parallel=False)
print(resultsCDF['t'][:].data)
print(resultsCDF.T)
print(resultsCDF.Nt)
print(resultsCDF.save_rate)
print(resultsCDF.backup_rate)
resultsCDF.close()

In [ ]:
previous_resultsCDF = Dataset(output_folder + '/results_testfb.nc', 'r', format='NETCDF4', parallel=False)
print(previous_resultsCDF['t'][:].data)
previous_resultsCDF.close()

## Launching the all simulation in one go to verify the results

In [ ]:
initialCDF = Dataset('initial.nc','r', format='NETCDF4', parallel=False)
verbose=1
mySim = Simulation(initialCDF,
                   methods, 
                   methods_kwargs,
                   output_folder,
                   verbose=verbose,
                   name='control')

In [ ]:
T = 8*3600
dt = 300
Nt = int(T//dt)
save_rate = 1
backup_rate = 10

mySim.run(T, Nt, save_rate, backup_rate, first_run=True)

The two simulation does not have the exact same save_rate (since we changed from 1 to 2 between the first and second part of the first simulation). Keep it in mind while comparing the results

In [ ]:
reference = Dataset(output_folder + '/results_control.nc', 'r', parallel=False)
perturbed = Dataset(output_folder + '/results_testfb_fb.nc', 'r', parallel=False)

In [ ]:
print(reference['t'][:].data)
print(perturbed['t'][:].data)

In [ ]:
k_per = 34
t = perturbed['t'][k_per]
k_ref = np.where(reference['t'][:].data == t)[0][0]

print("t = {}".format(t))
print("k_ref = {}".format(k_ref))

In [ ]:
variable = 'theta_t'
np.min(np.equal(perturbed[variable][:,:,k_per], reference[variable][:,:,k_ref]))

True means that all the values (on the spatial grid) of the variable are equals at instant t

In [ ]:
reference.close()
perturbed.close()